In [1]:
import pandas as pd
from pandas.plotting import table
import requests
import json
from config import key, bkey
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cbook, ticker, units
import datetime
import functools
import re

## US Census Data API

In [2]:
census = pd.read_html('https://api.census.gov/data.html')[0]

In [3]:
list(census["Title"])

['1986 County Business Patterns: Business Patterns',
 '1987 County Business Patterns: Business Patterns',
 '1988 County Business Patterns: Business Patterns',
 '1989 County Business Patterns: Business Patterns',
 '1990 County Business Patterns: Business Patterns',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: County Estimates by 5 Year Age Groups, Race by Sex, and Hispanic Origin',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: United States Civilian Population Estimates by Age and Sex',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: United States Resident plus Armed Forces Overseas Population Estimates by Age and Sex',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: United States Resident Population Estimates by Age and Sex',
 '1991 County Business Patterns: Business Patterns',
 '1992 County Business Patterns: Business Patterns',
 '1993 County Business Patterns: Business Patterns',
 '1994 County Business Patterns: Busines

### International Trade

In [4]:
trade_census = census[census["Title"].str.contains("International Trade")]
pd.set_option('display.max_colwidth', None)
trade_census

,Title,Description,Vintage,Dataset Name,Dataset Type,Geography List,Variable List,Group List,Examples,Developer Documentation,API Base URL
306,2014 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2014,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2014/intltrade/imp_exp
352,2015 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2015,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2015/intltrade/imp_exp
378,2016 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2016,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2016/intltrade/imp_exp
452,2017 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2017,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2017/intltrade/imp_exp
483,2018 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2018,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2018/intltrade/imp_exp
528,Time Series Economic Indicators Time Series -: U.S. International Trade in Goods and Services,"The U.S. Census Bureau.s economic indicator surveys provide monthly and quarterly data that are timely, reliable, and offer comprehensive measures of the U.S. economy. These surveys produce a variety of statistics covering construction, housing, international trade, retail trade, wholesale trade, services and manufacturing. The survey data provide measures of 

### Time Series International Trade: Monthly U.S. Imports by End-use Code <br>
The Census data API provides access to the most comprehensive set of data on current month and cumulative year-to-date imports using the End-use classification system. The End-use endpoint in the Census data API also provides value, shipping weight, and method of transportation totals at the district level for all U.S. trading partners.

### Time Series International Trade: Monthly U.S. Imports by Port and Harmonized System (HS) Code<br>
The Port HS endpoint in the Census data API also provides value, shipping weight, and method of transportation totals at the port level for all U.S. trading partners. The Census data API will help users research new markets for their products, establish pricing structures for potential export markets, and conduct economic planning.

#### Variables

In [5]:
portVar = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/porths/variables.html')[0]
portVar

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIR_VAL_MO,15-digit Air Value,Measures,not required,NaN,0,int,NaN,NaN
1,AIR_VAL_YR,15-digit Year-to-Date Air Value,Measures,not required,NaN,0,int,NaN,NaN
2,AIR_WGT_MO,15-digit Air Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
3,AIR_WGT_YR,15-digit Year-to-Date Air Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
4,CNT_VAL_MO,15-digit Containerized Vessel Value,Measures,not required,NaN,0,int,NaN,NaN
5,CNT_VAL_YR,15-digit Year-to-Date Containerized Vessel Value,Measures,not required,NaN,0,int,NaN,NaN
6,CNT_WGT_MO,15-digit Containerized Vessel Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
7,CNT_WGT_YR,15-digit Year-to-Date Containerized Vessel Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
8,COMM_LVL,4-character aggregation levels for commodity code. HS2=2-digit HS totals. HS4=4-digit HS totals. HS6=6-digit HS totals.,Measures,not required,NaN,0,string,NaN,NaN
9,CTY_CODE,4-character Country Code,Measures,default displayed,NaN,0,string,NaN,NaN


In [6]:
portDict = dict(zip(portVar["Name"], portVar["Label"].str.replace('15-digit ', '')))

In [7]:
portURL = 'https://api.census.gov/data/timeseries/intltrade/imports/porths?get='
portURLreq = requests.get(f"{portURL}PORT,PORT_NAME,CNT_WGT_MO&time=from+2019+to+2021&key={key}")
print(portURLreq.url)

https://api.census.gov/data/timeseries/intltrade/imports/porths?get=PORT,PORT_NAME,CNT_WGT_MO&time=from+2019+to+2021&key=c140aee8a215a9f809ef67c3c758614e79362587


In [8]:
portData = portURLreq.json()
portDF = pd.DataFrame(portData, columns=portData[0]).drop([0]).rename(columns=portDict)
pd.set_option('display.max_colwidth', None)
portDF

,4-character Port Code,150-character Port Name,Containerized Vessel Shipping Weight,ISO-8601 Date/Time value
1,-,TOTAL FOR ALL PORTS,15840254477,2019-01
2,-,TOTAL FOR ALL PORTS,13329969180,2019-02
3,-,TOTAL FOR ALL PORTS,14054279566,2019-03
4,-,TOTAL FOR ALL PORTS,14721360090,2019-04
5,-,TOTAL FOR ALL PORTS,15441730719,2019-05
...,...,...,...,...
9030,2401,"YSLETA, TX",0,2020-10
9031,2401,"YSLETA, TX",0,2020-11
9032,2401,"YSLETA, TX",0,2020-12
9033,2401,"YSLETA, TX",0,2021-01


In [9]:
YRportDF = portDF[~portDF['150-character Port Name'].str.contains("TOTAL")].drop(columns=['4-character Port Code']).rename(columns={'150-character Port Name':'Port','ISO-8601 Date/Time value':'Date'})
YRportDF['Date'] = pd.to_datetime(YRportDF['Date'])

In [10]:
YRportDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9008 entries, 27 to 9034
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Port                                  9008 non-null   object        
 1   Containerized Vessel Shipping Weight  9008 non-null   object        
 2   Date                                  9008 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 281.5+ KB


In [11]:
portCOL = YRportDF.columns.drop(['Port',"Date"])
YRportDF[portCOL] = YRportDF[portCOL].apply(pd.to_numeric, errors='coerce')
YRportDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9008 entries, 27 to 9034
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Port                                  9008 non-null   object        
 1   Containerized Vessel Shipping Weight  9008 non-null   int64         
 2   Date                                  9008 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 281.5+ KB


In [12]:
YRportDF.head()

,Port,Containerized Vessel Shipping Weight,Date
27,"PORTLAND, ME",8582227,2019-01-01
28,"PORTLAND, ME",14823829,2019-02-01
29,"PORTLAND, ME",12000667,2019-03-01
30,"PORTLAND, ME",19453602,2019-04-01
31,"PORTLAND, ME",14481568,2019-05-01


### Date

In [ ]:
Date = YRportDF.groupby(['Date','Port']).sum().nlargest(1000, ['Containerized Vessel Shipping Weight'], keep='all').unstack()
Date.dropna(axis='columns')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
Date.plot(ax=ax, legend=False)

In [ ]:
import seaborn as sns; sns.set_theme()

In [ ]:
YRportDF.info()

In [ ]:
corr = YRportDF.corr()
corr

In [ ]:
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
p = YRportDF.copy().rename(columns={'Containerized Vessel Shipping Weight': 'Weight'})
p

In [ ]:
df = p.pivot(index="Date", columns="Port", values="Weight").reset_index().rename_axis(None,axis=1)
df

In [ ]:
nan_value = float("NaN")
cleandf = df.replace(0, nan_value, inplace=True)
cleandf.dropna(how='all', axis=1, inplace=True)
cleandf

In [ ]:
df

In [ ]:
cleandf

In [ ]:
portVar = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/usda/variables.html')[0]
portVar

In [ ]:
portURL = 'https://api.census.gov/data/timeseries/intltrade/imports/usda?get='
portURLreq = requests.get(f"{portURL}COMM_LVL,CNT_VAL_MO,USDA,SUMMARY_LVL,SUMMARY_LVL2&time=2020&key={key}")
print(portURLreq.url)

In [ ]:
portData = portURLreq.json()
portDF = pd.DataFrame(portData, columns=portData[0]).drop([0]).rename(columns=portDict)
pd.set_option('display.max_colwidth', None)
portDF

#### Variables

In [ ]:
import_var = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/enduse/variables.html')[0]
import_var

In [ ]:
print(list(import_var['Name']))

In [ ]:
var_dict = dict(zip(import_var["Name"], import_var["Label"].str.replace('15-digit ', '')))

#### Data

In [ ]:
url = 'https://api.census.gov/data/timeseries/intltrade/imports/enduse?get='

Monthly Data

In [ ]:
mResponse = requests.get(f"{url}I_ENDUSE,I_ENDUSE_LDESC,I_ENDUSE_SDESC,DISTRICT,DIST_NAME,CC_MO,CNT_WGT_MO,VES_CHA_MO,CON_VAL_MO&time=from+2019+to+2021&key={key}")
print(mResponse.url)

In [ ]:
mTrade_data = mResponse.json()
mTrade = pd.DataFrame(mTrade_data, columns=mTrade_data[0]).drop([0]).rename(columns=var_dict)
pd.set_option('display.max_colwidth', None)
mTrade

In [ ]:
mTrade = mTrade[~mTrade['100-character Import ENDUSE Long Description'].str.contains("TOTAL")].drop(columns=['1- or 5-character Import ENDUSE Code','100-character Import ENDUSE Long Description','2-character District Code']).rename(columns={'100-character Import ENDUSE Short Description':'Product', '50-character District name': 'District', 'ISO-8601 Date/Time value':'Date'})
mTrade.info()

In [ ]:
mTrade = mTrade[~mTrade['District'].str.contains("TOTAL")]
mTrade.info()

In [ ]:
cols = mTrade.columns.drop(['Product','District','Date'])
mTrade[cols] = mTrade[cols].apply(pd.to_numeric, errors='coerce')
mTrade.dtypes

In [ ]:
mTrade['Date'] = pd.to_datetime(mTrade['Date'])
mTrade.dtypes

In [ ]:
mTrade.describe().astype(int)

In [ ]:
mTrade.astype('float64',errors='ignore').to_html(float_format=lambda x: format(x,',.2f'))

In [ ]:
mTrade.sort_values(by='Containerized Vessel Shipping Weight', ascending=True)

In [ ]:
Import_Consumption_Value = mTrade[mTrade['Imports for Consumption, Total Value']>1]
Import_Consumption_Value.head()

In [ ]:
Import_Consumption_Value = Import_Consumption_Value.sort_values(by='Imports for Consumption, Total Value', ascending=True)
Import_Consumption_Value.head().reset_index()

In [ ]:
Vessel_Charges = mTrade[mTrade['Vessel Charges']>1]
Vessel_Charges.head()

In [ ]:
Vessel_Charges = Vessel_Charges.sort_values(by='Imports for Consumption, Total Value', ascending=True)
Vessel_Charges.head().reset_index()

In [ ]:
Cont_Shipping_Weight = mTrade[mTrade['Containerized Vessel Shipping Weight']<0]
Cont_Shipping_Weight

In [ ]:
Cont_Shipping_Weight = Cont_Shipping_Weight.sort_values(by='Imports for Consumption, Total Value', ascending=True)
Cont_Shipping_Weight.head().reset_index()

In [ ]:
data = {"Cont_Shipping_Weight": mTrade['Imports for Consumption, Total Value'],
    "Vessel_Charges": mTrade['Containerized Vessel Shipping Weight']}

index = mTrade['Date']

In [ ]:
dataFrame   = pd.DataFrame(data=data);

normalized  = dataFrame.div(dataFrame.sum(axis=1), axis=0);

normalized.index = index;

print(normalized);

normalized.plot.area(stacked=True, figsize=(16,8));

plt.show();

In [ ]:
Product.sort_values(by='Containerized Vessel Shipping Weight', ascending=False)

In [ ]:
Product = mTrade['Product']
Container_Vessel_Weight = mTrade['Containerized Vessel Shipping Weight']
Vessel_Charges = mTrade["Vessel Charges"]
Consumption_Value = mTrade["Imports for Consumption, Total Value"]

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
# people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(Product))
# performance = 3 + 10 * np.random.rand(len(people))

ax.barh(y_pos, Vessel_Charges, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Performance')
ax.set_title('How fast do you want to go today?')

plt.show()

In [ ]:
df = df.rename(columns={'types':'count'})

In [ ]:
df

In [ ]:
df = mTrade.groupby(['Date','Product']).sum()

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(people))
performance = 3 + 10 * np.random.rand(len(people))
error = np.random.rand(len(people))

ax.barh(y_pos, performance, xerr=error, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Performance')
ax.set_title('How fast do you want to go today?')

plt.show()

In [ ]:
df

In [ ]:
mTrade.groupby('Product').cumsum()

In [ ]:
mTrade.groupby('District').cumsum()

<br>
Year Data

In [ ]:
Yresponse = requests.get(f"{url}I_ENDUSE,I_ENDUSE_LDESC,I_ENDUSE_SDESC,DISTRICT,DIST_NAME,CNT_CHA_YR,CNT_VAL_YR,CNT_WGT_YR,CON_VAL_YR,VES_CHA_YR,VES_VAL_YR,VES_WGT_YR&time=from+2010+to+2021&key={key}")
print(Yresponse.url)

In [ ]:
Ydf = Yresponse.json()
Ydf = pd.DataFrame(Ydf, columns=Ydf[0]).drop([0]).rename(columns=var_dict)
Ydf.head()

In [ ]:
Yvessel = Ydf[~Ydf['100-character Import ENDUSE Long Description'].str.contains("TOTAL")].drop(columns=['1- or 5-character Import ENDUSE Code','100-character Import ENDUSE Long Description','2-character District Code']).rename(columns={'100-character Import ENDUSE Short Description':'Product', '50-character District name': 'District', 'ISO-8601 Date/Time value':'Date'})
Yvessel.head()

In [ ]:
Product = Mvessel.groupby('District').mean()
Product

In [ ]:
Mvessel.plot.area(stacked=False, figsize=(14,7))

In [ ]:
# subjects
Scontainer = ['Containerized Vessel Charges', 'Containerized Vessel Value', 'Containerized Vessel Shipping Weight']
Sconsumption = ['Imports for Consumption, Charges', 'Imports for Consumption, Total Value']
Svessel = ['Vessel Charges', 'Vessel Value', 'Vessel Shipping Weight']

In [ ]:
ProductConsumption = Mvessel.drop(columns=Scontainer).drop(columns=Svessel)

ProductConsumption.plot.area(stacked=False, figsize=(14,7))

In [ ]:
ProductConsumption.plot.bar()

### Time Series International Trade: Monthly U.S. Imports by Port and Harmonized System (HS) Code
The Port HS endpoint in the Census data API also provides value, shipping weight, and method of transportation totals at the port level for all U.S. trading partners.

#### Variable

In [ ]:
port_var = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/porths/variables.html')[0]
print(list(port_var['Name']))

In [ ]:
var_dict2 = dict(zip(port_var["Name"], port_var["Label"].str.replace('15-digit ', '')))

#### Data

In [ ]:
url2 = ("https://api.census.gov/data/timeseries/intltrade/imports/porths?get=")

In [ ]:
response2 = requests.get(f"{url2}PORT,PORT_NAME,CNT_VAL_MO,CNT_WGT_MO,CTY_CODE,CTY_NAME,GEN_VAL_MO,VES_VAL_MO,VES_WGT_MO&time=2021&key={key}")
print(response2.url)

In [ ]:
df2 = response2.json()
df2 = pd.DataFrame(df2, columns=df2[0]).drop([0]).rename(columns=var_dict2)
df2.head()

In [ ]:
remove = df2[~df2["150-character Port Name"].str.contains("TOTAL")].drop(columns=['4-character Port Code','4-character Country Code'])
port = remove[~remove["50-character Country Name"].str.contains("TOTAL")].rename(columns={'150-character Port Name':'Port', '50-character District name': 'District', 'ISO-8601 Date/Time value':'Date'})
port.set_index('Port').head()

In [ ]:
import requests
import json
import prettytable

In [ ]:
bls_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [ ]:
bls_response = requests.get("")